In [1]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [4]:
# 모델 초기화 (temperature=0.5로 설정해 일관성 있는 출력)
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

### 문제 2-1 : 콤마 구분 리스트 파서 활용

In [5]:
# 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

In [6]:
# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()

In [7]:
# 사용자가 관심있는 분야(예: "음식", "스포츠", "영화" 등)를 입력하면,
# 해당 분야와 관련된 한국의 유명한 장소나 활동 5가지를 콤마로 구분된
# 리스트로 출력하는 프로그램을 작성하세요.

In [8]:
# 프롬프트 템플릿 설정
template = """
{subject}와(과) 관련된 한국의 유명한 장소나 활동 5가지를 콤마로 구분해서 알려주세요.

{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

In [9]:
# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

In [10]:
user_input = input("관심있는 분야를 입력하세요(예. 음식, 스포츠 등):")
result = chain.invoke({"subject": user_input})
print(result)

['서울의 한옥마을', '부산의 자갈치 시장', '전주 한옥마을', '경주의 황남빵 마을', '남해의 독일마을']


### 문제 2-2 : 영화 리뷰 감정 분석기

In [19]:
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException
from langchain_core.prompts import ChatPromptTemplate

from enum import Enum
from pprint import pprint
import os

In [20]:
class ReviewSentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

In [21]:
# EnumOutputParser 초기화
parser = EnumOutputParser(enum=ReviewSentiment)
format_instructions = parser.get_format_instructions()

In [22]:
# 프롬프트 템플릿
template = """
당신은 텍스트 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:"""

In [23]:
prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)   

In [24]:
# OutputFixingParser로 안정성 향상
fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)

In [28]:
texts = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",
    "스토리가 지루하고 전개가 느렸어요.",
    "음향과 영상미가 뛰어나서 몰입감이 좋았습니다."
]

In [29]:
# 안전한 감정 분석 함수 (에러 처리 포함)
def safe_sentiment_analysis(text, use_fixing_parser=True):
    """안전한 감정 분석 함수 - 에러 처리 포함"""
    try:
        # 기본 체인 생성
        chain = prompt | model | (fixing_parser if use_fixing_parser else parser)
        
        # 분석 실행
        result = chain.invoke({"text": text})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

In [30]:
# 실제 감정 분석 실행
def run_sentiment_analysis():
    """실제 감정 분석 실행"""
    print("=== 실제 감정 분석 결과 ===")
    
    success_count = 0
    total_count = len(texts)
    
    for i, text in enumerate(texts, 1):
        print(f"\n{i}. 텍스트: {text}")
        
        # OutputFixingParser 사용
        result, error = safe_sentiment_analysis(text, use_fixing_parser=True)
        
        if result:
            print(f"   감정: {result.value} ")
            success_count += 1
        else:
            print(f"   오류: {error} ")
            
            # 기본 파서로 재시도
            print("   기본 파서로 재시도...")
            result2, error2 = safe_sentiment_analysis(text, use_fixing_parser=False)
            
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
                success_count += 1
            else:
                print(f"   재시도 실패: {error2} ")
    
    print(f"\n=== 결과 요약 ===")
    print(f"성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
    print(f"실패: {total_count-success_count}/{total_count}")

# 실제 분석 실행 (API 키가 있는 경우)
try:
    run_sentiment_analysis()
except Exception as e:
    print("API 키가 설정되지 않았거나 네트워크 오류:")
    print("실제 실행을 위해서는 OpenAI API 키를 설정하세요.")
    print(f"오류 상세: {e}")

=== 실제 감정 분석 결과 ===

1. 텍스트: 이 영화 정말 재미없어요. 시간 낭비였습니다.
   감정: 부정 

2. 텍스트: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
   감정: 긍정 

3. 텍스트: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
   감정: 보통 

4. 텍스트: 스토리가 지루하고 전개가 느렸어요.
   감정: 부정 

5. 텍스트: 음향과 영상미가 뛰어나서 몰입감이 좋았습니다.
   감정: 긍정 

=== 결과 요약 ===
성공: 5/5 (100.0%)
실패: 0/5


### 문제 2-3: 학생 정보 구조화 시스템

In [31]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

In [34]:
# 출력 구조 정의
class StudentClass(BaseModel):
    name: str = Field(description="이름")
    age: int = Field(description="나이")
    major: str = Field(description="전공")
    hobbies: List[str] = Field(description="취미 리스트")
    goal: str = Field(description="목표")

In [35]:
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=StudentClass)

In [36]:
# 프롬프트 템플릿 설정
template = """
학생의 자유 형식 자기소개를 입력받아 이름, 나이, 전공, 취미 리스트, 목표를 구조화된 형태로 보여주세요.
요청: {introduce}

{format_instructions}
"""

In [37]:
prompt = ChatPromptTemplate.from_template(template)
# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)


In [38]:
introduce =  "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
chain = prompt | model | parser
output = chain.invoke({"introduce": introduce})

In [39]:
# 결과 출력
print(output)

name='김민수' age=22 major='컴퓨터공학' hobbies=['게임하기', '영화보기', '코딩'] goal='훌륭한 개발자가 되는 것'


### 문제 2-4 : 여행 계획 분석기

In [41]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from pprint import pprint

In [43]:
# 출력 구조 정의 (평점, 장점, 단점, 요약)
response_schemas = [
    ResponseSchema(name="destination", description="여행 계획의 목적지"),
    ResponseSchema(name="duration", description="여행 기간 (예 2박 3일)"),
    ResponseSchema(name="budget", description="여행 비용 예산"),
    ResponseSchema(name="rating", description="여행 계획 추천도 5점 만점에서 평점"),
    ResponseSchema(name="activities", description="계획들을 리스트로 출력")
]

In [44]:
# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

In [45]:
# 프롬프트 템플릿
template = """
다음 여행 리뷰를 분석하세요. 리뷰 내용: {review}

{format_instructions}
"""

In [46]:
prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

In [47]:
# 테스트 리뷰 데이터
review = """
지난 주에 부산으로 2박 3일 여행을 다녀왔어요.
총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고,
감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다.
5점 만점에 4점 정도 줄 수 있을 것 같아요.
"""

In [48]:
# 체인 실행
chain = prompt | model | parser

output = chain.invoke({"review": review})

In [49]:
# 결과 출력 (Pretty Print)
pprint(output)

{'activities': '해운대에서 바다구경, 자갈치시장에서 회 먹기, 감천문화마을 구경',
 'budget': '30만원',
 'destination': '부산',
 'duration': '2박3일',
 'rating': '4점'}
